<a href="https://colab.research.google.com/github/rhkrtpgus/TIL/blob/main/2023_01_27ddareungi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 라이브러리 로딩

In [ ]:
pip install scikit-learn

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns

In [ ]:
from sklearn.ensemble import RandomForestClassifier

## 2.데이터 로딩 밑 확인

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/All-in/train.csv')
test = pd.read_csv('/content/drive/MyDrive/All-in/test.csv') 
submission = pd.read_csv('/content/drive/MyDrive/All-in/submission.csv')

* id: 고유 id
* hour: 시간
* temperature: 기온
* precipitation: 비가 오지 않았으면 0, 비가 오면 1
* windspeed: 풍속(평균)
* humidity: 습도
* visibility: 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
* ozone: 오존
* pm10: 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
* pm2.5: 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
*count: 시간에 따른 따릉이 대여 수


In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
print(train.columns.values)

In [ ]:
corrmat = train.corr(method='spearman')
indexes = corrmat.nlargest(n=11, columns='count').index
corrmat = train[indexes].corr(method='spearman')
fig, ax = plt.subplots()
fig.set_size_inches(15,15)
sns.heatmap(corrmat,annot=True, square=True, cmap ='Reds')

### 상관관계
: 두 변수 중에서 한 변수의 변화가 다른 변수의 변화에 따라 어떤 변화가 일어나는지를 보여주는 지표입니다.

> 상관계수는 -1.00에서 +1.00 사이의 값을 가집니다
> 양의 상관관계일 경우에는 (+)값이 나타나고, 음의 상관관계일 경우에는(-) 값이 나타납니다. 양의 상관관계는 한 변수가 증가함에 따라 다른 변수도 증가하는 경우를 말하며, 음의 상관관계는 한 변수가 증가함에 따라 다른 변수는 감소하는 경우를 말합니다

>* +- 0.9 이상일때 : 상관관계가 아주 높다
>* +- 0.7~0.9일때: 상관관계가 있다
>* +- 0.4~0.7 이상일때 : 상관관계가 있다
>* +- 0.2~0.4 이상일때 : 상관관계가 있으나 매우 낮다
>* +- 0.2 미만일때 : 상관관계가 거의 없다

### 상관관계가 있는 변수
> 결국 사람들이 변수에 따라 따릉이를 얼마나 이용하는가를 예측하기 때문에 count 변수가 중요하게 작용할거라고 생각했다.
> count 변수와 관련이 높은 변수는 hour, hour_bef_temperature, hour_bef_windspread, hour_bef_ozone, hour_bef_humidity 정도였다.
> 그 외에도 관심을 줄만한 정도의 상관관계가 나온 두 변수에는
>* hour_bef_pm10, hour_bef_2.5 사이의 상관관계 => hour_bef_visivility와 상관관계를 나타낸다
>* hour_bef_windspread, hour_bef_ozone 사이의 상관관계
>* hour_bef_precipitation, hour_bef_ozone사이의 상관관계
> 등등이 있지만 따릉이 사용량을 예측하는데 유의미한 결과가 있을것같지 않았다



##결측치 확인 밑 처리

In [ ]:
# null값 처리하기
train.isnull().sum()

In [ ]:
test.isnull().sum()

> hour, hour_bef_temperature, hour_bef_windspread, hour_bef_ozone, hour_bef_humidity, count 정도가 사용하려고 하는 변수인데, test세트와 train세트에 결측치가 있는 것을 확인할 수있었다. 

> 우선, 시간에는 결측치가 없다. 기온에는 평균을, precipitation에는 모두 0을 넣어주도록 하겠다. 풍속은 최빈값을 사용할 예정이다. 마지막으로 오존인데, 오존은 너무 많은 결측치가 있어 유의미한 변수로 써야할지 고민을 좀 해봐야할 것 같다.

In [ ]:
#train.fillna(0,inplace = True)
#test.fillna(0,inplace = True)

# 수정 전 코드: 모든 결측치를 0으로 처리해서 나중에 모델들을 구축해보았을때나, 시각화를 위해 그래프를 그렸을때 제대로 된 결과값이 나오지 않는 것을 확인할 수 있었다.

In [ ]:
# 기온에 평균넣기
train['hour_bef_temperature'] = train['hour_bef_temperature'].fillna(value = train['hour_bef_temperature'].mean())

In [ ]:
train.isnull().sum() # 기온의 결측치가 0이 된것을 확인 할 수 있다 

In [ ]:
# 비가 오면 1, 아니면 0 의 결측치 전부 0 넣어주기
train['hour_bef_precipitation'] = train['hour_bef_precipitation'].fillna(0)

In [ ]:
train.isnull().sum() 

In [ ]:
# 풍속에 최빈값 넣기
print(stats.mode('hour_bef_windspeed'))

In [ ]:
print("최빈값:", train['hour_bef_windspeed'].mode()[0])

In [ ]:
train['hour_bef_windspeed'] = train['hour_bef_windspeed'].fillna(2)

In [ ]:
train.isnull().sum() 

In [ ]:
# 습도 평균과 최빈값보기
print("최빈값:", train['hour_bef_humidity'].mode()[0])

print("평균값:", train['hour_bef_humidity'].mean())

최빈값과 평균 둘 중 어느것을 넣어야할지 고민하다가 둘다 넣어보았는데.. 생각 보다 차이가 컸다. 그럼 최빈값은 전체 CSV 셀 중 몇개를 차지해서 최빈값이 되었는지가 궁금해져 찾아보았다.

In [ ]:
#from collections import Counter

#Counter(train['F'])
#Counter(train['F'])['44']

In [ ]:
train.count()

In [ ]:
train['hour_bef_humidity'].count()

In [ ]:
h= train['hour_bef_humidity'].value_counts()
print(h)

In [ ]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf) # 저 ... 없애고 싶었다

In [ ]:
h= train['hour_bef_humidity'].value_counts()
print(h) # 안사라진다

In [ ]:
pd.set_option('display.max_rows',10000) # 보이는 max row의 값을 10000으로 설정

In [ ]:
h= train['hour_bef_humidity'].value_counts()
print(h) # 야호

In [ ]:
# 8번 이하로 나온 수들은 모두 없애고 최빈값으로 대체 후 평균 구하기

df= pd.DataFrame(h)

In [ ]:
# 16 바꾸기
df.loc[157,'F']=44
df.loc[444,'F']=44
df.loc[643,'F']=44
df.loc[787,'F']=44
df.loc[852,'F']=44
df.loc[1194,'F']=44
df.loc[1199,'F']=44
df.loc[1281,'F']=44

In [ ]:
# 88 바꾸기
df.loc[346,'F']=44
df.loc[61,'F']=44
df.loc[509,'F']=44
df.loc[715,'F']=44
df.loc[1016,'F']=44
df.loc[1311,'F']=44
df.loc[1402,'F']=44
df.loc[1415,'F']=44

In [ ]:
# 89 바꾸기

df.loc[2,'F']=44
df.loc[263,'F']=44
df.loc[338,'F']=44
df.loc[607,'F']=44
df.loc[800,'F']=44
df.loc[1417,'F']=44
df.loc[1445,'F']=44

In [ ]:
# 99 바꾸기

df.loc[118,'F']=44
df.loc[224,'F']=44
df.loc[460,'F']=44
df.loc[541,'F']=44
df.loc[889,'F']=44
df.loc[1052,'F']=44
df.loc[1097,'F']=44

In [ ]:
# 17 바꾸기

df.loc[343,'F']=44
df.loc[390,'F']=44
df.loc[146,'F']=44
df.loc[102,'F']=44
df.loc[887,'F']=44
df.loc[1202,'F']=44
df.loc[1212,'F']=44

In [ ]:
# 92 바꾸기   
df.loc[171,'F']=44
df.loc[181,'F']=44
df.loc[308,'F']=44
df.loc[548,'F']=44
df.loc[694,'F']=44
df.loc[1307,'F']=44

# 18 바꾸기   
df.loc[33,'F']=44
df.loc[243,'F']=44
df.loc[337,'F']=44
df.loc[839,'F']=44
df.loc[909,'F']=44
df.loc[1392,'F']=44

# 12 바꾸기  
df.loc[348,'F']=44
df.loc[711,'F']=44
df.loc[812,'F']=44
df.loc[958,'F']=44
df.loc[1093,'F']=44
df.loc[1333,'F']=44

# 19 바꾸기
df.loc[22,'F']=44
df.loc[69,'F']=44
df.loc[290,'F']=44
df.loc[360,'F']=44
df.loc[580,'F']=44

# 98 바꾸기 
df.loc[103,'F']=44
df.loc[668,'F']=44
df.loc[1111,'F']=44
df.loc[1233,'F']=44
df.loc[147,'F']=44

# 11 바꾸기   
df.loc[11,'F']=44
df.loc[949,'F']=44
df.loc[998,'F']=44
df.loc[1176,'F']=44

# 94 바꾸기
df.loc[311,'F']=44
df.loc[497,'F']=44
df.loc[816,'F']=44
df.loc[1290,'F']=44

# 15 바꾸기 
df.loc[1050,'F']=44
df.loc[1123,'F']=44
df.loc[1428,'F']=44

# 13 바꾸기  
df.loc[258,'F']=44
df.loc[908,'F']=44

# 93 바꾸기    
df.loc[130,'F']=44
df.loc[638,'F']=44

# 8 바꾸기     
df.loc[341,'F']=44

# 10 바꾸기    
df.loc[410,'F']=44

# 95 바꾸기   
df.loc[438,'F']=44

# 7 바꾸기     
df.loc[6,'F']=44

# 9 바꾸기  
df.loc[573,'F']=44

In [ ]:
pd.set_option('display.max_rows',15) # 보이는 max row의 값을 15으로 설정

In [ ]:
print("평균값:", train['hour_bef_humidity'].mean()) # 안변했다..?

In [ ]:
train['hour_bef_humidity'] = train['hour_bef_humidity'].fillna(52.23129718599863)

## 3.데이터 시각화
>##### * 시간대별 자전거 이용
>##### * 비가 올때의 자전거 이용
>##### * 미세먼지 pm10일때의 자전거 이용
>##### * 미세먼지가 pm2.5일때의 자전거 이용


In [ ]:
# 시간대 별 자전거 이용
train.groupby(['hour'])['count'].mean()

In [ ]:
# 시간대 별 자전거 이용 시각화 # import mataplotlib,pyplot as plt
train.groupby(['hour'])['count'].mean().plot()

##### 다른 분이 하신걸 찾아보니까 그래프에 글자도 넣을 수 있었다. 출근시간과 퇴근시간에 확실히 사용량이 많은 것을 한 눈에 알아 볼 수 있게 한번 글씨도 넣어봤다.

##### Q. 어떤 값들을 넣어도 x,y축 구간은 알아서 정해주는건지...?

In [ ]:
train.groupby(['hour'])['count'].mean().plot()
plt.axvline(x=8,color='r')
plt.axvline(x=18,color='r') # x축 8, 18에 빨간 선 넣어주기
plt.text(6, 150, 'go to work')
plt.text(16, 150, 'leave work') # (6,150), (16,150) 지점에 각각 출근, 퇴근 글씨 넣어주기

# 한글은 문자로 입력되지 않음
# 글자를 입력하면 좌표를 시작으로 생기기 때문에 중간에 입력하려면 하나씩 넣어봐야함

>###### * groupby() : 데이터를 그룹별로 분할하여 독립된 그룹에 대하여 별도로 데이터를 처리(혹은 적용)하거나 그룹별 통계량을 확인하고자 할 때 유용한 함수이다

In [ ]:
# 비가 올 때의 자전거 이용
train.groupby(['hour_bef_precipitation'])['count'].mean()

In [ ]:
train.groupby(['hour_bef_precipitation'])['count'].mean().plot()

##### 해보고 이건 아니다 싶어서 막대 그래프로 비가 오지 않았으면 0, 비가 오면 1에 대한 수치를 시각화 했다.
##### 찾아보니 막대 그래프 함수는 Matplotlib에서 bar()함수를 기본적으로 사용한다고 하여 사용했다.

In [ ]:
# train.groupby(['hour_bef_precipitation'])['count'].mean().plt.bar()
# 에러났고요..

In [ ]:
x = np.arange(2)
rain = ['rain', 'no rain'] # rain = 1, no rain = 0
count = [' 33.673913', '111.001415']

plt.bar(x, count)
plt.xticks(x, rain)

plt.show()

>##### 그래도 비가 오지 않을때 사람들이 따릉이를 훨씬 더 많이 사용한다는 것을 알 수 있다.
>##### Q. 비가 올때의 그래프가 아예 보이지 않는데 이거 보이게 하려면 어떻게...?


In [ ]:
# 미세먼지 pm10일때의 자전거 이용(미세먼지지)
pm10 =train.groupby(['hour_bef_pm10'])['count'].mean()
print(pm10)

In [ ]:
# 미세먼지 pm2.5일때의 자전거 이용(초미세먼지)
pm25 =train.groupby(['hour_bef_pm2.5'])['count'].mean()
print(pm25)

In [ ]:
# 미세먼지 pm10일때의 자전거 이용 시각화
train.groupby(['hour_bef_pm10'])['count'].mean().plot()

In [ ]:
# 미세먼지 pm2.5일때의 자전거 이용 시각화화
train.groupby(['hour_bef_pm2.5'])['count'].mean().plot()

In [ ]:
pm25.mean()

In [ ]:
pm10.mean()

두개의 그래프를 놓고 봤을때는 y 축의 단위가 달라 잘 몰랐지만, 두개의 평균을 비교해보니 유의미한 차이는 없는것같다. 사람들이 미세먼지, 초미세 먼지 농도에 따라 상관없이 거의 동일하게 따릉이를 이용한 것같다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pm=['pm2.5', 'pm10']
#df = pd.DataFrame[{'hour_bef_pm2.5' : pm25, 'hour_bef_pm10':pm10}, index= pm]

# 바 굵기 조정
#bar_width = 0.25

# 미세먼지가 pm2.5일때, pm10일때 2가지 경우
#index=np.arange(2)

#pm1 = plt.bar(index, train['hour_bef_pm2.5'], bar_width, alpha=0.4, color='blue', label='pm2.5') #미세먼지가 pm2.5일때
#pm2 = plt.bar(index, train['hour_bef_pm10'], bar_width, alpha=0.4, color='red', label='pm10') #미세먼지가 pm10일때

# x축, y축 이름 및 범례 설정
#plt.xlabel('pm_concentration', size = 13)
#plt.ylabel('count', size = 13)
#plt.legend()
#plt.show()

# 결측치 때문일 수도 있다, 평균으로 null 처리 해보기기

In [ ]:
# 습도랑 따릉이 횟수와의 관계

train.groupby(['hour_bef_humidity'])['count'].mean().plot()

## 시각화 결론
>##### 따릉이는 사람들이 출근하고 퇴근 하는 시간인 7시와 17시 에 가장 많이 사용 되었으며, 비가 오지 않을 때는 비가 올때보다 3배 이상의 따릉이가 사용 됨을 알 수 있다. 또한, 미세먼지와 초미세먼지 농도는 사람들이 따릉이를 이용하는데 유의미한 차이를 주지 못했다.

## 4. 변수 선택 및 모델 구축

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import math
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
# 필요 없는 변수 잘라주기
df2 = train.drop(['id','hour_bef_visibility','hour_bef_ozone','hour_bef_pm10','hour_bef_pm2.5'],inplace= True, axis=1)

In [ ]:
# 잘 잘렸나 확인하기
train.columns

* KNN모형

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
#model_dict = {'DT':DecisionTreeRegressor(),
#             'RF':RandomForestRegressor(),
#             'LGB':lgb.LGBMRegressor(),
#             'XGB':xgb.XGBRegressor(),
#            'KNN':KNeighborsRegressor()}

In [ ]:
#def min_max_normalize(list):
#
#    normalized = []
#
#    for value in list:
#      normalized_num = (value - min(lst)) / (max(lst) - min(lst))
#      normalized.append(normalized_num)
#
#    return normalized

In [ ]:
#from sklearn.model_selection import KFold

In [ ]:
#k_fold = KFold(n_splits=5, shuffle= True, random_state=10)

In [ ]:
#from sklearn.model_selection import cross_val_score

In [ ]:
#score = {}
#
#for model_name in model_dict.keys():
#    
#    model = model_dict[model_name]
#    
#    score[model_name] = np.mean(cross_val_score(model, train, train, scoring = 'neg_mean_squared_error', n_jobs = -1, cv = k_fold))
#   
#    print(f'{model_name} 평가 완료')

In [ ]:
#pd.Series(score).plot(kind = 'bar') # 다른분거 따라해봤는데... 왜 되는걸까요?

분명 막 점수가 줄줄 나와서 거기서 한 2,3등짜리 선택한것같은데 왜....안나올까요?!?!?!

##구글링 해서 찾아본 방법

In [ ]:
col_names = ['hour_bef_temperature', 'hour_bef_precipitation', 'hour_bef_windspeed', 'hour_bef_humidity', 'count']
dataset = pd.read_csv('/content/drive/MyDrive/All-in/train.csv', encoding='UTF-8')

print(dataset.info())

In [ ]:
print(dataset.iloc[0:5]) # dataset.head()
print(dataset.iloc[-5:]) # dataset.tail()

In [ ]:
X = dataset.iloc[:,:-1].to_numpy()

y = dataset.iloc[:,4].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(len(X_train), len(X_test))

In [ ]:
print(X_train[:3])
print(y_train[:3])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # Scaler 객체 생성
scaler.fit(X_train) # 스케일링(표준화)를 위한 평균과 표준 편차 계산
X_train = scaler.transform(X_train) # 스케일링(표준화 수행)
X_test = scaler.transform(X_test)

밑의 코드에서 평균과 표준편차가 nan이 나와서 쭉 에러가 나는 것 같은데 이건 어떻게 없앨까요?

In [ ]:
for col in range(4):
    print(f'평균 = {X_train[:, col].mean()}, 표준편차= {X_train[:, col].std()}')

In [ ]:
print(f'평균 = {X_train[:, 2].mean()}, 표준편차= {X_train[:, 2].std()}')

In [ ]:
print(f'평균 = {X_train[:, 3].mean()}, 표준편차= {X_train[:, 3].std()}')

In [ ]:
train.isnull().sum()

In [ ]:
for col in range(4):
    print(f'평균 = {X_test[:, col].mean()}, 표준편차= {X_test[:, col].std()}')

In [ ]:
# 학습/예측
#from sklearn.neighbors import KNeighborsClassifier
#
#classifier = KNeighborsClassifier(n_neighbors=5)
#classifier.fit(X_train, y_train)

In [ ]:
#y_pred= classifier.predict(X_test)
#print(y_pred)

In [ ]:
#  모델 평가

#from sklearn.metrics import confusion_matrix
#conf_matrix= confusion_matrix(y_test, y_pred)
#print(conf_matrix)

In [ ]:
#from sklearn.metrics import classification_report
#report = classification_report(y_test, y_pred)
#print(report)

In [ ]:
# 모델 개선 - k값을 변화시킬 때, 에러가 줄어드는 지
#import numpy as np
#    
#for i in range(1, 31):
#    knn = KNeighborsClassifier(n_neighbors = i)
#    knn.fit(X_train, y_train)
#    pred_i = knn.predict(X_test)
#    errors.append(np.mean(pred_i != y_test))
#print(errors)

다른 방법

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
test['hour_bef_temperature']=test['hour_bef_temperature'].fillna(test['hour_bef_temperature'].mean())
test['hour_bef_precipitation']=test['hour_bef_precipitation'].fillna(0)
test['hour_bef_windspeed']=test['hour_bef_windspeed'].fillna(2.4)
test['hour_bef_humidity']=test['hour_bef_humidity'].fillna(52)
test['hour_bef_visibility']=test['hour_bef_visibility'].fillna(2000)
test['hour_bef_ozone']=test['hour_bef_ozone'].fillna(0.0411)
test['hour_bef_pm10']=test['hour_bef_pm10'].fillna(test['hour_bef_pm10'].mean())
test['hour_bef_pm2.5']=test['hour_bef_pm2.5'].fillna(test['hour_bef_pm2.5'].mean())

In [ ]:
test.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print('X_train shape:', X_train.shape)
print('X_valid shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_valid shape:', y_test.shape)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)

mean_squared_error = metrics.mean_squared_error(y_hat, y_test)
knn_rmse = math.sqrt(mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % knn_rmse)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
print(acc_knn)

* 로지스틱 회귀 분석

* Support Vector Machines

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
print(acc_svc)

* perception

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
print(acc_perceptron)

* 결정트리

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
print(acc_decision_tree)

* liner Regression

## 5.모델 테스트

In [ ]:
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
test['hour_bef_visibility'].mode()

In [ ]:
test['hour_bef_visibility'].mean()

In [ ]:
v= test['hour_bef_visibility'].value_counts()
print(v)

In [ ]:
test['hour_bef_ozone'].mode()

In [ ]:
test['hour_bef_ozone'].mean()

In [ ]:
o= test['hour_bef_ozone'].value_counts()
print(o)

In [ ]:
test['hour_bef_temperature']=test['hour_bef_temperature'].fillna(test['hour_bef_temperature'].mean())
test['hour_bef_precipitation']=test['hour_bef_precipitation'].fillna(0)
test['hour_bef_windspeed']=test['hour_bef_windspeed'].fillna(test['hour_bef_windspeed'].mode())
test['hour_bef_humidity']=test['hour_bef_humidity'].fillna(52)
test['hour_bef_visibility']=test['hour_bef_visibility'].fillna(2000)
test['hour_bef_ozone']=test['hour_bef_ozone'].fillna(0.0411)
test['hour_bef_pm10']=test['hour_bef_pm10'].fillna(test['hour_bef_pm10'].mean())
test['hour_bef_pm2.5']=test['hour_bef_pm2.5'].fillna(test['hour_bef_pm2.5'].mean())

In [ ]:
test.isnull().sum()

In [ ]:
test['hour_bef_windspeed'].mode()

In [ ]:
test['hour_bef_temperature']=test['hour_bef_temperature'].fillna(test['hour_bef_temperature'].mean())
test['hour_bef_precipitation']=test['hour_bef_precipitation'].fillna(0)
test['hour_bef_windspeed']=test['hour_bef_windspeed'].fillna(2.4)
test['hour_bef_humidity']=test['hour_bef_humidity'].fillna(52)
test['hour_bef_visibility']=test['hour_bef_visibility'].fillna(2000)
test['hour_bef_ozone']=test['hour_bef_ozone'].fillna(0.0411)
test['hour_bef_pm10']=test['hour_bef_pm10'].fillna(test['hour_bef_pm10'].mean())
test['hour_bef_pm2.5']=test['hour_bef_pm2.5'].fillna(test['hour_bef_pm2.5'].mean())

In [ ]:
test.isnull().sum()

In [ ]:
model = 
model.fit(X, y)
Y_pred = model.predict(test)

In [ ]:
Y_pred